<a href="https://colab.research.google.com/github/RockhoRockho/Data_Visualization/blob/main/_Bokeh_%ED%95%9C%EB%B2%88%EC%97%90_%EC%A0%9C%EB%8C%80%EB%A1%9C_%EB%B0%B0%EC%9A%B0%EA%B8%B03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bokeh 한번에 제대로 배우기


## Bokeh 특징

* 최신 브라우저의 인터랙티브 시각화
* 독립형 HTML 문서 또는 서버 지원
* 표현력이 뛰어나고 다양한 그래픽 지원
* 큰 동적 데이터 또는 스트리밍 데이터 지원
* 파이썬(또는 Scala, R, ...)에서 쉽게 사용
* 자바스크립트 불필요

In [1]:
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, show
output_notebook()

* 샘플 데이터 다운로드

In [2]:
import bokeh.sampledata
bokeh.sampledata.download()

Creating /root/.bokeh directory
Creating /root/.bokeh/data directory
Using data directory: /root/.bokeh/data
Downloading: CGM.csv (1589982 bytes)
   1589982 [100.00%]
Downloading: US_Counties.zip (3171836 bytes)
   3171836 [100.00%]
Unpacking: US_Counties.csv
Downloading: us_cities.json (713565 bytes)
    713565 [100.00%]
Downloading: unemployment09.csv (253301 bytes)
    253301 [100.00%]
Downloading: AAPL.csv (166698 bytes)
    166698 [100.00%]
Downloading: FB.csv (9706 bytes)
      9706 [100.00%]
Downloading: GOOG.csv (113894 bytes)
    113894 [100.00%]
Downloading: IBM.csv (165625 bytes)
    165625 [100.00%]
Downloading: MSFT.csv (161614 bytes)
    161614 [100.00%]
Downloading: WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.zip (4816256 bytes)
   4816256 [100.00%]
Unpacking: WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.csv
Downloading: gapminder_fertility.csv (64346 bytes)
     64346 [100.00%]
Downloading: gapminder_population.csv (94509 bytes)
     94509 [100.00%]
Downloading: gapminder_life_e

### 색상 막대(Color bars)

In [5]:
from bokeh.models import ColumnDataSource
from bokeh.sampledata.autompg import autompg
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import transform

source = ColumnDataSource(autompg)
color_mapper = LinearColorMapper(palette='Inferno256',
                                 low=autompg.weight.min(),
                                 high=autompg.weight.max())
p = figure(x_axis_label='Horse Power', y_axis_label='MPG',
           tools='', toolbar_location=None)
p.circle(x='hp', y='mpg',
         color=transform('weight', color_mapper),
         size='cyl', alpha=0.5, source=autompg)
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=10,
                     location=(0, 0), title='Weight')
p.add_layout(color_bar, 'right')
show(p)

## 레이아웃(Layout)

In [6]:
from bokeh.layouts import row

x = list(range(10))
y1 = np.random.randint(1, 10, 10)
y2 = np.random.randint(1, 10, 10)
y3 = np.random.randint(1, 10, 10)

s1= figure(width=250, plot_height=250)
s1.square(x, y1, size=10, color='red', alpha=0.5)
s2= figure(width=250, plot_height=250)
s2.square(x, y2, size=10, color='green', alpha=0.5)
s3= figure(width=250, plot_height=250)
s3.square(x, y3, size=10, color='blue', alpha=0.5)

show(row(s1, s2, s3))

In [7]:
from bokeh.layouts import gridplot

p = gridplot([[s1, s2], [s3, None]], toolbar_location=None)
show(p)

## 연결된 상호작용(Linked Interactions)

In [10]:
plot_options = dict(width=250, plot_height=250, tools='pan, wheel_zoom')

s1 = figure(**plot_options)
s1.square(x, y1, size=10, color='red', alpha=0.5)
s2= figure(x_range=s1.x_range, **plot_options)
s2.square(x, y2, size=10, color='green', alpha=0.5)
s3= figure(y_range=s1.y_range, **plot_options)
s3.square(x, y3, size=10, color='blue', alpha=0.5)

p = gridplot([[s1, s2, s3]])
show(p)

In [11]:
x = list(range(-20, 21))
y1, y2 = [abs(i) for i in x], [i ** 2 for i in x]

source = ColumnDataSource(data=dict(x=x, y1=y1, y2=y2))
tools = 'box_select, lasso_select, help'

left = figure(tools=tools, width=300, height=300)
left.circle('x', 'y1', source=source)
right = figure(tools=tools, width=300, height=300)
right.circle('x', 'y2', source=source)

p = gridplot([[left, right]])
show(p)

In [13]:
from bokeh.models import HoverTool

source = ColumnDataSource(
    data=dict(
        x=np.random.randint(1, 10, 5),
        y=np.random.randint(1, 10, 5),
        desc=['A', 'B', 'C', 'D', 'E']
    )
)

hover = HoverTool(
    tooltips=[
              ('index', '$index'),
              ("(x, y)", "($x, $y)"),
              ("desc", "@desc")
    ]
)

p = figure(plot_width=300, plot_height=300, tools=[hover])
p.circle('x', 'y', size=20, source=source)
show(p)

## 위젯(Widgets)

In [18]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider

x = [x*0.005 for x in range(0, 201)]
source = ColumnDataSource(data=dict(x=x, y=x))

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

slider = Slider(start=0.1, end=8, value=1, step=.1, title='power')

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
  var data = source.data;
  var f = slider.value;
  var x = data['x'];
  var y = data['y'];
  for (var i = 0; i < x.length; i++) {
    y[i] = Math.pow(x[i], f)
  }
  
  source.change.emit();
""")
slider.js_on_change('value', update_curve)

show(column(slider, plot))

In [28]:
from bokeh.layouts import column, row
from bokeh.models import CustomJS, Slider
from bokeh.plotting import ColumnDataSource, figure, show

x = np.linspace(0, 10, 500)
y = np.sin(x)
source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(y_range=(-10, 10), plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title='Amplitutde')
freq_slider = Slider(start=0.1, end=10, value=1, step=.1, title='Frequency')
phase_slider = Slider(start=0, end=6.4, value=0, step=.1, title='Phase')
offset_slider = Slider(start=-5, end=5, value=0, step=.1, title='Offset')

callback = CustomJS(args=dict(source=source,
                              amp=amp_slider, freq=freq_slider,
                              phase=phase_slider, offset=offset_slider),
                    code="""
    const data = source.data;
    const A = amp.value;
    const k = freq.value;
    const phi = phase.value;
    const B = offset.value;
    const x = data['x']
    const y = data['y']
    for (var i = 0; i < x.length; i++) {
      y[i] = B + A * Math.sin(k * x[i] + phi);
    }
    source.change.emit();
""")
amp_slider.js_on_change('value', callback)
freq_slider.js_on_change('value', callback)
phase_slider.js_on_change('value', callback)
offset_slider.js_on_change('value', callback)

layout = row(
    plot, 
    column(amp_slider, freq_slider, phase_slider, offset_slider)
)

show(layout)

In [30]:
from random import random

x = [random() for x in range(200)]
y = [random() for y in range(200)]
color = ['blue'] * len(x)

s1 = ColumnDataSource(data=dict(x=x, y=y, color=color))
p = figure(plot_width=400, plot_height=400, tools='lasso_select')
p.circle('x', 'y', color='color', size=8, alpha=0.4, source=s1,
         selection_color='orange', selection_alpha=0.4)

s2 = ColumnDataSource(data=dict(xm=[0,1], ym=[0.5, 0.5]))
p.line(x='xm', y='ym', color='red', line_width=5, alpha=0.6, source=s2)

callback = CustomJS(args=dict(s1=s1, s2=s2), code="""
  var inds = s1.selected.indices;
  if (inds.length == 0)
    return;

  var ym = 0
  for (var i = 0; i < inds.length; i++) {
    ym += s1.data.y[inds[i]]
  }

  ym /= inds.length
  s2.data.ym = [ym, ym]
  s2.change.emit();
""")

s1.selected.js_on_change('indices', callback)

show(p)

## 막대와 범주형 데이터 플롯(Bar and Categorical Data Plots)

In [33]:
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6

cities = ['서울특별시', '부산광역시', '인천광역시',
          '대구광역시', '대전광역시', '광주광역시']
pops = [9720846, 3404423, 2947217, 2427954, 1471040, 1455048]
source = ColumnDataSource(data=dict(cities=cities, pops=pops, color=Spectral6))

p = figure(x_range=cities, plot_height=300, y_range=(0, 10000000), title='Populations')
p.vbar(x='cities', top='pops', width=0.7, color='color', legend_field='cities', source=source)
p.xgrid.grid_line_color = None
p.legend.orientation = 'vertical'
p.legend.location = 'top_right'

show(p)

In [36]:
from bokeh.palettes import GnBu3, PuBu3

years = ['2018', '2019', '2020']
cities = ['서울특별시', '부산광역시', '인천광역시',
          '대구광역시', '대전광역시', '광주광역시']
male = {'cities' : cities,
        '2018' : [-4802769, -1700822, -1481769, -1224126, -747071, -723562],
        '2019' : [-4762711, -1683596, -1482934, -1212450, -740834, -722559],
        '2020' : [-4732274, -1668618, -1476813, -1198815, -734441, -720060]}
female = {'cities' : cities,
        '2018' : [5011280, 1754789, 1472114, 1245491, 747807, 737183],
        '2019' : [4994433, 1744036, 1474090, 1238028, 742104, 736465],
        '2020' : [4988571, 1735805, 1470404, 1229139, 736599, 734988]}
        
p = figure(y_range=cities, plot_height=250, x_range=(-16000000, 16000000))
p.hbar_stack(years, y='cities', height=0.8, color=GnBu3, source=ColumnDataSource(male),
             legend_label=['%s 남자' % x for x in years])
p.hbar_stack(years, y='cities', height=0.8, color=PuBu3, source=ColumnDataSource(female),
             legend_label=['%s duwk' % x for x in years])
p.y_range.range_padding = 0.1
p.ygrid.grid_line_color= None
p.legend.location = 'top_right'

show(p)

In [38]:
from bokeh.models import FactorRange

x = [ (city, year) for city in cities for year in years ]
pops = sum(zip(female['2018'], female['2018'], female['2020']), ())
source = ColumnDataSource(data=dict(x=x, pops=pops))

p = figure(x_range=FactorRange(*x), plot_height=250, title='Female Population by Year')
p.vbar(x='x', top='pops', width=0.8, source=source)
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

In [40]:
from bokeh.transform import factor_cmap

p = figure(x_range=FactorRange(*x), plot_height=250, title='Female Population by Year')
p.vbar(x='x', top='pops', width=0.8, source=source, line_color='white',
       fill_color=factor_cmap('x', palette=['skyblue', 'royalblue', 'darkblue'],
                              factors=years, start=1, end=2))
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

In [42]:
from bokeh.sampledata.commits import data
from bokeh.transform import jitter

days = ['Sun', 'Sat', 'Fri', 'Thu', 'Wed', 'Tue', 'Mon']
source = ColumnDataSource(data)

p = figure(plot_width=800, plot_height=300,
           y_range=days, x_axis_type='datetime')
p.circle(x='time', y=jitter('day', width=0.6, range=p.y_range),
         color='orange', source=source, alpha=0.4)
p.xaxis[0].formatter.days = ['%Hh']
p.x_range.range_padding = 0
p.ygrid.grid_line_color = None

show(p)

## 내보내기(Exporting)

In [44]:
from bokeh.sampledata.stocks import AAPL, IBM, MSFT, GOOG
from bokeh.palettes import Spectral4

p = figure(plot_width=800, plot_height=250, x_axis_type='datetime')

for data, name, color in zip([AAPL, IBM, MSFT, GOOG], ["AAPL", "IBM", "MSFT", "GOOG"], Spectral4):
  datetime = np.asarray(data['date'], dtype=np.datetime64)
  value = np.asarray(data['close'])
  p.line(datetime, value, line_width=1, color=color, alpha=0.8, legend_label=name)

p.legend.location = 'top_left'
p.legend.click_policy='hide'

show(p)

In [45]:
from bokeh.io import output_file, show

output_file('stocks.html')
show(p)

In [46]:
!ls

sample_data  stocks.html


In [47]:
!cat stocks.html





<!DOCTYPE html>
<html lang="en">
  
  <head>
    
      <meta charset="utf-8">
      <title>Bokeh Plot</title>
      
      
        
          
        
        
          
        <script type="text/javascript" src="https://cdn.bokeh.org/bokeh/release/bokeh-2.3.3.min.js" integrity="sha384-dM3QQsP+wXdHg42wTqW85BjZQdLNNIXqlPw/BgKoExPmTG7ZLML4EGqLMfqHT6ON" crossorigin="anonymous"></script>
        <script type="text/javascript">
            Bokeh.set_log_level("info");
        </script>
        
      
      
    
  </head>
  
  
  <body>
    
      
        
          
          
            
              <div class="bk-root" id="e47e6316-4dca-46d3-a9d1-6d61006eb381" data-root-id="7260"></div>
            
          
        
      
      
        <script type="application/json" id="8324">
          {"7b25bc20-b32a-4529-a392-dd190258b6d1":{"defs":[],"roots":{"references":[{"attributes":{"below":[{"id":"7269"}],"center":[{"id":"7272"},{"id":"7276"},{"id":"7319"}],"height":250,"left

## 참고 문헌

* Bokeh, https://bokeh.org/